In [4]:
from pathlib import Path
import pandas as pd
import json
import datetime
import numpy as np
from tabulate import tabulate

# Aggregated Data Transactions

In [ ]:
def Data_Transactions():
    root_dir = Path("../data/aggregated/transaction/country/india/state/")
    lis = []
    # rglob is recursive glob - which means searching files recursiverly inside the files and folders
    for i in root_dir.rglob(f"*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            # i will have exact path from that <.parent> is going one step behind and <.name> is getting that subdirectory name
            state_name = i.parent.parent.name
            year = i.parent.name

            # to get the filename 1.json,2.json, 3.json and 4.json 
            quarter = i.stem
            # to change 1.json to Q1, 2.json to Q2 etc
            quarter = f'Q{quarter}'

            lis.append({'quarter':quarter,'year':year,'state':state_name,'data':dataset})

    # print(lis)

    lis2=[]
    for j in lis:
        
        for k in j['data']['data']['transactionData']:
            # I'm doing conversion rounding to two decimal point
            amount_value = k['paymentInstruments'][0]['amount']
            amount_formatted = f'{amount_value: .2f}'

            timestamp_milliseconds = j['data']['responseTimestamp']
            timestamp_seconds = timestamp_milliseconds /1000
            datetime_obj = datetime.datetime.fromtimestamp(timestamp_seconds)
            formatted_timestamp = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')

            
            data = dict(quarter = j['quarter'],
                        year = j['year'],
                        state = j['state'],
                        name = k['name'],
                        type = k['paymentInstruments'][0]['type'],
                        count = k['paymentInstruments'][0]['count'],
                        amount = amount_formatted,
                        timestamp = formatted_timestamp)
            lis2.append(data)

    return lis2
    

In [ ]:
dataTransaction = pd.DataFrame(Data_Transactions())
dataTransaction['year'] = dataTransaction['year'].astype(int)
dataTransaction['amount'] = dataTransaction['amount'].astype(float)
# dataTransaction['country'] = 'India'
dataTransaction.info()


In [ ]:
import mysql.connector
from tabulate import tabulate

In [ ]:
def MyCursor():
    # Database Connection
    db = mysql.connector.connect(
        host ='localhost',
        user = 'root',
        password ='balaji',
        database = 'pulse'
    )

    mycursor = db.cursor(buffered=True)
    mycursor
    db.commit()

    return db, mycursor

In [ ]:
def aggDataTransTable():
    db, mycursor = MyCursor()
    mycursor.execute("create database if not exists pulse")
    mycursor.execute("show databases")
    # mycursor.execute("drop table aggDataTrans")
    mycursor.execute("""create table if not exists aggDataTrans (
                    aggDataTransId int auto_increment primary key,quarter varchar(10), year int(10), 
                    state varchar(255), name varchar(255), type varchar(10), count bigint, amount float,
                    timestamp datetime)
                    """)
    sql = ("""insert into aggDataTrans(aggDataTransId ,quarter ,year ,state ,name, type, count, amount, 
            timestamp) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)
            on duplicate key update
            quarter = values(quarter), year = values(year), state = values(state), name = values(name), 
            type = values(type), count = values(count), amount = values(amount),
            timestamp = values(timestamp)""")

    for i in dataTransaction.to_records().tolist():
        mycursor.execute(sql,i)

    db.commit()
    mycursor.close()
    db.close()
    



In [ ]:
aggDataTransTable()

In [ ]:
import numpy as np

In [ ]:
def aggDataTrans(QUARTER, YEAR):
    db,mycursor = MyCursor()

    mycursor.execute("""select
    quarter,
    state,
    year,
    sum(count) as All_Transactions, 
    concat(round(sum(amount)/ 10000000)) as Total_Payment_Value,
    round(sum(amount)/sum(count)) as Avg_Transactions_Value
    from aggdatatrans
    group by quarter, year, state;""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'state', 'year', 'All_Transactions', 'Total_Payment_Value', 'Avg_Transactions_Value']

    DataTransaction = pd.DataFrame(data, columns=columns)
    DataTransaction = DataTransaction[(DataTransaction['quarter'] == QUARTER) & (DataTransaction['year'] == YEAR)]
    DataTransaction['Total_Payment_Value'] = DataTransaction['Total_Payment_Value'].astype(int)
    DataTransaction['All_Transactions'] = DataTransaction['All_Transactions'].astype(np.int64)

    with open('statesmapping.json', 'r') as file:
        state_mapping = json.load(file)

    # Use the map function to replace old values with new values
    DataTransaction['state'] = DataTransaction['state'].map(state_mapping)

    return DataTransaction

In [ ]:
result = aggDataTrans('Q3', 2023)
result

In [2]:
import mysql.connector
db = mysql.connector.connect(
    host ='localhost',
    user = 'root',
    password ='balaji',
    database = 'pulse'
)

mycursor = db.cursor(buffered=True)
mycursor
db.commit()

In [73]:
def aggDataTransCategory(QUARTER, YEAR):
    db,mycursor = MyCursor()
    # Payment categories 
    mycursor.execute("""select 
    quarter, 
    year, 
    name as Transactions_Name,
    sum(count) as All_Trans,
    sum(amount) as Total_Pay_value
    from aggdatatrans
    group by quarter, year, name ;""")

    out = mycursor.fetchall()
    data = list(out)

    columns = ['quarter', 'year', 'Transactions_Name', 'Transcations', 'Payment_Value']

    DataTransactionName = pd.DataFrame(data, columns=columns)
    DataTransactionName['Transcations'] = DataTransactionName['Transcations'].astype(np.int64)
    DataTransactionName['Payment_Value'] = np.int64(round(DataTransactionName['Payment_Value'],0))
    DataTransactionName = DataTransactionName[(DataTransactionName['quarter'] == QUARTER) & (DataTransactionName['year'] == YEAR)]


    DataTransactionName['Total_Transactions'] = sum(DataTransactionName['Transcations'])
    DataTransactionName['Total_Payment_Value'] = np.int64(round(sum(DataTransactionName['Payment_Value'])/10000000,0))
    DataTransactionName['Avg_Transaction_Value'] = np.int64(round(sum(DataTransactionName['Payment_Value']) / sum(DataTransactionName['Transcations']),0))

    return DataTransactionName

In [76]:
result_category = aggDataTransCategory('Q3', 2023)

In [81]:
result_category

,quarter,year,Transactions_Name,Transcations,Payment_Value,Total_Transactions,Total_Payment_Value,Avg_Transaction_Value
109,Q3,2023,Merchant payments,9868263225,4686919160817,17041723189,2401213,1409
110,Q3,2023,Peer-to-peer payments,5929633156,18342575171224,17041723189,2401213,1409
111,Q3,2023,Recharge & bill payments,1229061495,967688175533,17041723189,2401213,1409
112,Q3,2023,Financial Services,6273407,8553147714,17041723189,2401213,1409
113,Q3,2023,Others,8491906,6397948031,17041723189,2401213,1409


In [84]:
result_category[['Transactions_Name', 'Transcations']]

,Transactions_Name,Transcations
109,Merchant payments,9868263225
110,Peer-to-peer payments,5929633156
111,Recharge & bill payments,1229061495
112,Financial Services,6273407
113,Others,8491906


In [ ]:
import plotly.graph_objects as go


In [ ]:
fig = go.Figure(data=go.Choropleth(
    geojson="India.geojson",
    featureidkey='properties.ST_NM',
    locationmode='geojson-id',
    locations=result['state'],
    customdata=result[['All_Transactions', 'Total_Payment_Value', 'Avg_Transactions_Value']],
    z=result['Total_Payment_Value'],
    hovertemplate="<b>%{location}</b><br>" +
                "All Transactions: %{customdata[0]:,.0f}<br>" +
                "Total Payment Value: %{customdata[1]:,.0f} Cr<br>" +
                "Average Transactions Value: %{customdata[2]:,.0f}",


    autocolorscale=False,
    colorscale='Reds',
    marker_line_color='peachpuff',

    colorbar=dict(
        title={'text': "Total_Payment_Value"},

        thickness=15,
        len=0.35,
        bgcolor='rgba(255,255,255,0.6)',

        tick0=0,
        dtick=20000,

        xanchor='left',
        x=0.01,
        yanchor='bottom',
        y=0.05
    )
))



fig.update_geos(
    visible=False,
    projection=dict(
        type='conic conformal',
        parallels=[12.472944444, 35.172805555556],
        rotation={'lat': 24, 'lon': 80}
    ),
    lonaxis={'range': [68, 98]},
    lataxis={'range': [6, 38]}
)

fig.update_layout(
    title=dict(
        text="PhonePe",
        xanchor='center',
        x=0.5,
        yref='paper',
        yanchor='bottom',
        y=1,
        pad={'b': 10}
    ),
    margin={'r': 0, 't': 30, 'l': 0, 'b': 0},
    height=550,
    width=550
)

fig.show()



# Aggregated Data Users

In [ ]:
def AggData_Users():
    root_dir1 = Path("../data/aggregated/user/country/india/state")

    lis = []
    for i in root_dir1.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name
            #  to get 1.josn,2.json,3.json and 4.json
            quarter = i.stem
            #  to convert 1.json to Q1, 2.json to Q2 etc.,
            quarter = f'Q{quarter}'

            lis.append({'quarter':quarter, 'year': year, 'state' : state, 'data': dataset})

    # print(lis)

    lis2 = []
    for j in lis:
        data = {
            'quarter': j['quarter'],
            'year': j['year'],
            'state': j['state'],
            # 'timestamp': j['data'].get('responseTimestamp'),
            'registeredUsers': j['data']['data']['aggregated'].get('registeredUsers', 'NaN'),
            'appOpens': j['data']['data']['aggregated'].get('appOpens', 'NaN')
            
        }

        # Check if 'usersByDevice' data is available
        users_by_device = j['data']['data']['usersByDevice']
        if users_by_device is not None:
            for user_data in users_by_device:
                brand = user_data.get('brand', 'NaN')
                count = user_data.get('count', 'NaN')
                percentage_value = user_data.get('percentage', None)
                percentage_formated = f'{percentage_value: .2f}' if percentage_value is not None else '0'
                lis2.append({
                    **data,  # Include quarter, year, and state in the data
                    'brand': brand,
                    'count': count,
                    'percentage': percentage_formated
                    
                })

        # If 'usersByDevice' is missing or empty, add a row with 'NaN' for brand, count, and percentage
        if not users_by_device:
            lis2.append({
                'quarter': data['quarter'],
                'year': data['year'],
                'state': data['state'],
                'registeredUsers': data['registeredUsers'],
                'appOpens': data['appOpens'],
                'brand': '0',
                'count': '0',
                'percentage': '0'
            })

    return lis2


In [ ]:
DataUsers = pd.DataFrame(AggData_Users())
DataUsers['year'] = DataUsers['year'].astype(int)
DataUsers['count'] = DataUsers['count'].astype(int)
DataUsers['percentage'] = DataUsers['percentage'].astype(float)
# DataUsers['timestamp'] = pd.to_datetime(DataUsers['timestamp'], unit='ms')
# DataUsers['timestamp'] = DataUsers['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
Data_Users = DataUsers

In [ ]:
def aggDataUsersTable():
    db, mycursor = MyCursor()
    mycursor.execute("""create table if not exists aggDataUsers (
                    aggDataUsersId int auto_increment primary key, quarter varchar(10), year int(10), 
                    state varchar(255), registeredUsers bigint, appOpens bigint,
                    brand varchar(255), count bigint, percentage float)""")
    
    sql = ("""insert into aggdatausers (aggDataUsersId, quarter, year, state, registeredUsers, appOpens, 
       brand, count, percentage) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update
       quarter = values(quarter), year = values(year), state = values(state), 
       registeredUsers = values(registeredUsers), appOpens = values(appOpens), 
       brand = values(brand), count = values(count), percentage = values(percentage)""")

    for i in Data_Users.to_records().tolist():
        mycursor.execute(sql,i)

    db.commit()
    mycursor.close()
    db.close()

In [ ]:
aggDataUsersTable()

In [ ]:
# mycursor.execute("describe aggDataUsers")

# out = mycursor.fetchall()
# print(tabulate(out, [i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
def aggDataUsers(QUARTER, YEAR):
    mycursor.execute("""select quarter, year, state, registeredUsers, appOpens 
                    from aggdatausers  
                    group by quarter,year,state;""")

    out = mycursor.fetchall()
    # print(tabulate(out, [i[0] for i in mycursor.description], tablefmt='psql'))

    data = list(out)
    columns = ['quarter', 'year', 'state', 'registeredUsers', 'appOpens']
    UsersData = pd.DataFrame(data, columns= columns)
    UsersData = UsersData[(UsersData['quarter']== QUARTER) & (UsersData['year'] == YEAR)]
    # UsersData

    with open("statesmapping.json", 'r') as file:
        state_mapping = json.load(file)

    UsersData['state']= UsersData['state'].map(state_mapping)

    return UsersData


In [ ]:
result = aggDataUsers('Q3', 2023)
# result

In [ ]:
import plotly.express as px

In [ ]:

# Create a custom hover template to display the desired columns
hover_template = "<b>%{location}</b><br>" + \
                "registeredUsers: %{customdata[0]:.0f}<br>" + \
                "appOpens: %{customdata[1]:.0f}"

fig = px.choropleth(
    result,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='registeredUsers',
    custom_data=['registeredUsers','appOpens'],
    hover_name= result.index,
    color_continuous_scale='Viridis',

)

# Set the hover template
fig.update_traces(
    hovertemplate=hover_template
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(geo=dict(
    center={'lon': 78.9629, 'lat': 20.5937},
    projection_scale=5))
fig.show()

In [35]:
def UsersCategory(QUARTER, YEAR):    
    mycursor.execute("""select
        quarter,
        year,
        sum(registeredUsers),
        sum(appOpens)
        from mapdatausers
        group by quarter, year""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'year', 'RegisteredUsers', 'AppOpens']

    def covert_crore_lakh(value):
        if value >= 10000000:
            return f'{value/10000000 : .7f} cr'
        else:
            return f'{value/100000 : .6f} L'
    UsersAppOpens = pd.DataFrame(data=data, columns=columns)
    UsersAppOpens = UsersAppOpens[(UsersAppOpens['quarter'] == QUARTER) & (UsersAppOpens['year'] == YEAR)]
    UsersAppOpens[['RegisteredUsers', 'AppOpens']] = UsersAppOpens[['RegisteredUsers', 'AppOpens']].applymap(covert_crore_lakh)
    UsersAppOpens['year'] = UsersAppOpens['year'].astype(str)
    return UsersAppOpens

In [36]:
UsersCategory('Q1', 2018)

,quarter,year,RegisteredUsers,AppOpens
0,Q1,2018,4.6877021 cr,0.000000 L


In [34]:
mycursor.execute("""select
    quarter,
    year,
    sum(registeredUsers),
    sum(appOpens)
    from mapdatausers
    group by quarter, year""")

out = mycursor.fetchall()
data = list(out)
columns = ['quarter', 'year', 'RegisteredUsers', 'AppOpens']

def covert_crore_lakh(value):
    if value >= 10000000:
        return f'{value/10000000 : .7f} cr'
    else:
        return f'{value/100000 : .6f} L'
UsersAppOpens = pd.DataFrame(data=data, columns=columns)
UsersAppOpens = UsersAppOpens[(UsersAppOpens['quarter'] == 'Q1') & (UsersAppOpens['year'] == 2018)]

UsersAppOpens[['RegisteredUsers', 'AppOpens']] = UsersAppOpens[['RegisteredUsers', 'AppOpens']].applymap(covert_crore_lakh)
UsersAppOpens['year'] = UsersAppOpens['year'].astype(str)
UsersAppOpens

,quarter,year,RegisteredUsers,AppOpens
0,Q1,2018,4.6877021 cr,0.000000 L
